# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [2]:
# All import statements needed for the project
import json
import pathlib
import urllib.parse
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import seaborn as sns
import folium
from shapely.geometry import Point

In [3]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"
NYC_DATA_APP_TOKEN = "SMCztjSAKApgTYc1CbAxcdksE"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [4]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [5]:
def download_nyc_geojson_data(url, force=False):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / url_path
    
    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        
        ...
        
        with open(filename, "w") as f:
            json.dump(..., f)
        print(f"Done downloading {url}.")

    else:
        print(f"Reading from {filename}...")

    return filename

In [6]:
# Read the ZIP code data file into a GeoDataFrame, keep the necessary columns: zipcode and geometry,
# and remove any rows that contain missing values. 
def load_and_clean_zipcodes():
    df = gpd.read_file(ZIPCODE_DATA_FILE)
    df['centroid'] = df.geometry.centroid
    df.geometry = df['centroid']
    columns_to_keep = ['ZIPCODE', 'geometry']
    df_subset = df[columns_to_keep].dropna()
    return df_subset

In [8]:
# Download and clean complaint 311 data
def download_and_clean_311_data():
    # Define date range for the SoQL query
    start_date = '2023-12-07T00:00:00'
    end_date = '2023-12-08T23:59:59'
    limit = 5000

    # Build the query URL with SoQL parameters
    base_url = "https://data.cityofnewyork.us/resource/erm2-nwe9.json"
    soql_query = f"?$where=created_date between '{start_date}' and '{end_date}'&$limit={limit}"
    url = base_url + soql_query
    
    headers = {
        'Accept': 'application/json', 
        'X-App-Token': NYC_DATA_APP_TOKEN  
    }

    response = requests.get(url, headers=headers)
    data = response.json()
    df = pd.json_normalize(data)

    # Keep necessary columns: unique key, date, zipcode, complaint type, latitude, and longitude
    # and create point column defined by longitude and latitude
    columns_to_keep = ['unique_key', 'created_date', 'incident_zip', 'complaint_type', 'latitude', 'longitude']
    df_subset = df[columns_to_keep].dropna()
    df_subset['created_date'] = pd.to_datetime(df_subset['created_date'])
    geometry = [Point(xy) for xy in zip(df_subset['longitude'], df_subset['latitude'])]
    df_subset = gpd.GeoDataFrame(df_subset, geometry=geometry, crs='EPSG:4326')

    return df_subset

In [ ]:
def download_and_clean_tree_data():
    raise NotImplementedError()

In [ ]:
def load_and_clean_zillow_data():
    raise NotImplementedError()

In [ ]:
def load_all_data():
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    df_zillow_data = load_and_clean_zillow_data()
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [ ]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

In [ ]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

In [ ]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

In [ ]:
geodf_311_data.info()

In [ ]:
geodf_311_data.head()

In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

In [ ]:
df_zillow_data.info()

In [ ]:
df_zillow_data.head()

## Part 2: Storing Data

In [ ]:
def setup_new_postgis_database(username, db_name):
    raise NotImplementedError()

In [ ]:
setup_new_postgis_database(DB_USER, DB_NAME)

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
TODO
"""

NYC_311_SCHEMA = """
TODO
"""

NYC_TREE_SCHEMA = """
TODO
"""

ZILLOW_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [ ]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    pass

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
def write_dataframes_to_table(tablename_to_dataframe):
    # write INSERT statements or use pandas/geopandas to write SQL
    raise NotImplemented()

In [ ]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [ ]:
write_dataframes_to_table(tablename_to_dataframe)

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)